<a href="https://colab.research.google.com/github/SatadruMukherjee/Data-Preprocessing-Models/blob/main/Receive_message_from_SQS_Queue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3

In [2]:
import json
import boto3
import datetime
from datetime import date

In [3]:
sqs_send= boto3.client('sqs',  aws_access_key_id='',
aws_secret_access_key='', region_name='us-east-2')

# **Send one message & try to read the same , ovserver the events in SQS Console**

In [4]:
# Defining Custom Function to Send Message to Main Queue and Deleting it from DLQ 
def sendQueue(sendQueueUrl,messageBody,messageAttributes) :
    # Calling Send Message API
    ret = sqs_send.send_message( QueueUrl=sendQueueUrl, 
                            MessageBody=messageBody,
                            MessageAttributes=messageAttributes) 
    return ret

In [5]:
sendQueueUrl=''

In [6]:
MessageAttributes={
    'Name': {
        'StringValue': 'Satadru Mukherjee',
        'DataType': 'String'
    },
    'Age': {
        'StringValue': '5',
        'DataType': 'Number'
    }}
messageBody='Hello World 10'
response = sendQueue(sendQueueUrl,messageBody,MessageAttributes)

In [7]:
messages = sqs_send.receive_message(QueueUrl=sendQueueUrl, MaxNumberOfMessages=10
                                    ,WaitTimeSeconds=20,MessageAttributeNames=['All'])

In [ ]:
messages

In [ ]:
receipthandle=messages['Messages'][0]['ReceiptHandle']
print(receipthandle)

# **Delete the messagea**

In [ ]:
sqs_send.delete_message(QueueUrl=sendQueueUrl, ReceiptHandle=receipthandle)  

# **Try to receive the message when the queue is empty**

In [11]:
messages = sqs_send.receive_message(QueueUrl=sendQueueUrl, MaxNumberOfMessages=10
                                    ,WaitTimeSeconds=20,MessageAttributeNames=['All'])

In [ ]:
messages

# **Note : the "Messages" key is not there when queue is empty**

In [13]:
def process_sqs_message():
  while True:
    messages = sqs_send.receive_message(QueueUrl=sendQueueUrl, MaxNumberOfMessages=10
                                      ,WaitTimeSeconds=20,MessageAttributeNames=['All'])
    if 'Messages' in messages:
      for m in messages['Messages']:
        print(m['Body'],m['ReceiptHandle'],m['MessageAttributes'])
        sqs_send.delete_message(QueueUrl=sendQueueUrl, ReceiptHandle=m['ReceiptHandle'])  
    else:
      print('Queue is currently Empty or Messages are Invisible')
      break

# **Test the above code**

In [14]:
MessageAttributes={
    'Name': {
        'StringValue': 'Satadru Mukherjee',
        'DataType': 'String'
    },
    'Age': {
        'StringValue': '5',
        'DataType': 'Number'
    }}
messageBody='Hello World 1'
response = sendQueue(sendQueueUrl,messageBody,MessageAttributes)
MessageAttributes={
    'Age': {
        'StringValue': '5',
        'DataType': 'Number'
    }}
messageBody='Hello World 2'
response = sendQueue(sendQueueUrl,messageBody,MessageAttributes)
MessageAttributes={
    'Name': {
        'StringValue': 'Satadru',
        'DataType': 'String'
    }}
messageBody='Hello World 3'
response = sendQueue(sendQueueUrl,messageBody,MessageAttributes)

In [ ]:
process_sqs_message()